In [1]:
import os
import getpass
import pytz
import json

from pathlib import Path

import pandas as pd
import numpy as np
import geopandas as gpd

from shapely.geometry import Point, Polygon
from shapely_geojson import dumps

from ipywidgets import Layout, Button, Output

from ipyleaflet import (
    GeoJSON, TileLayer, WidgetControl,
    Marker
)

from traitlets import (
    Int, Bool, Unicode, link
)
import ipyvuetify as v

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as m;
from sepal_ui.scripts import utils as su
from sepal_ui.frontend.styles import *

from component.scripts.scripts import *
from component.widget.custom_widgets import *
from component.frontend.styles import *
from component.message import cm


Styles()

ResizeTrigger()

HTML(value='\n<style>\nbody.jp-Notebook, \ndiv.jp-Cell,\ndiv.jp-OutputArea-output {\n\n    margin: 0 !importan…

In [2]:
COUNTRIES = gpd.read_file('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json')

In [77]:
class DynamicSelect(v.Card):
    
    """ Widget to navigate with next and previous buttons over a list
    
    Args:
        items (list) : List of items to be displayed in select list
        label (str) : Label to display into widget
        
    Parameters:
        v_model (traitlets.Any): Current element from select list
        
    Example:
        
        [1] ds = DynamicSelect(items=[1,2,3,4,5])
            ds # Display Dynamic select widget
        
        [2] # add behaviour once v_model changes
            ds.observe(lambda x: print(x), 'v_model')
    """
    
    items = List([]).tag(sync=True)
    v_model = Any().tag(sync=True)
    confidence = Unicode('All').tag(sync=True)
    
    def __init__(self, label='', **kwargs):
        
        self.class_='d-flex align-center mb-2'
        self.row=True
        self.label = label
        
        super().__init__(**kwargs)
        
        self.w_prev = v.Btn(
            _metadata = {'name':'previous'},
            x_small=True, 
            children=[
                v.Icon(left=True,children=['mdi-chevron-left']),
                'prev'
            ])

        self.w_next = v.Btn(
            _metadata = {'name' : 'next'},
            x_small=True, 
            children=[
                v.Icon(children=['mdi-chevron-right']),
                'nxt'
            ])
        self.w_conf = v.Select(
            class_='ma-2', 
            label='Confidence', 
            v_model='All', 
            items=['All', 'Low','High', 'Nominal']
        )
        
        self.w_list = v.Select(
            class_='ma-2',
            label=self.label, 
            items=self.items,
            v_model=''
        )

        self.children = [
            self.w_prev,
            self.w_conf,
            self.w_list,
            self.w_next
        ]
        
        link((self.w_list, 'items'),(self, 'items'))
        link((self.w_list, 'v_model'),(self, 'v_model'))
        link((self.w_conf, 'v_model'),(self, 'confidence'))
        
        self.w_prev.on_event('click', self.prev_next_event)
        self.w_next.on_event('click', self.prev_next_event)
        
    def prev_next_event(self, widget, change, data):
        
        current = self.w_list.v_model
        position = -1 if not current else self.w_list.items.index(current)
        last = len(self.w_list.items) - 1
            
        if widget._metadata['name']=='next':
            if position < last:
                self.w_list.v_model = self.w_list.items[position+1]

        elif widget._metadata['name']=='previous':
            if position > 0:
                self.w_list.v_model = self.w_list.items[position-1]

In [91]:
class UI(v.Layout):
    
    USER = getpass.getuser()
    TIME_SPAN = ['24 hours', '48 hours', '7 days']
    
    timespan = Unicode('24 hours').tag(sync=True)
    cloud_cover = Int(20).tag(sync=True)
    days_before = Int(0).tag(sync=True)
    max_images = Int(6).tag(sync=True)
    api_key = Unicode('').tag(sync=True)
    
    def __init__(self, **kwargs):
        
        self.class_='pa-2'

        
        super().__init__(**kwargs)
        
        # Start workspace
        self.root_dir, self.data_dir = self._workspace()
        
        self.alerts = None
        self.aoi = None
        self.aoi_alerts = None
        self.current_alert = None
        self.client = None
        self.valid_api = False
        self.lat = None
        self.lon = None
        
        # Instantiante map
        self.map_ = m.SepalMap(basemaps=['Google Satellite'], dc=True)
        self.map_.show_dc()

        # Widgets
        
        self.w_api_key = PasswordField(
            label=cm.ui.insert_api,
            v_model=self.api_key
        )
        self.w_api_btn = sw.Btn('Check ', small=True,)
        
        self.w_spantime = v.Select(
            label="In the last",
            items=self.TIME_SPAN,
            v_model=self.timespan,
        )
        
        self.w_aoi_method = v.Select(
            label=cm.ui.aoi_method,
            v_model='Draw on map',
            items=['Draw on map', 'Select country'],
            
        )
        self.w_countries = v.Select(
            label="Select country",
            v_model='',
            items=COUNTRIES.name.to_list(),
        )
        
        self.w_alert = Alert()

        self.w_days_before = NumberField(
            label=cm.ui.days_before,
            max_=5,
            v_model=self.days_before,
            disabled=True
        )
        
        self.w_max_images = NumberField(
            label=cm.ui.max_images,
            max_=6,
            min_=1,
            v_model=1,
            disabled=True
        )
        
        self.w_cloud_cover = v.Slider(
            label=cm.ui.cloud_cover,
            thumb_label=True,
            v_model=self.cloud_cover,
            disabled=True
        )

        self.w_alerts = DynamicSelect(disabled=True)
        
        self.reload_btn = Button(
            disabled=False,
            tooltip='Reload Planet imagery',
            icon='refresh',
            layout=Layout(width='30px', height='30px', line_height='30px', padding='0px')
        )
        
        # Create output space for metadata
        self.metadata_output = Output()
        
        # Add metadata_output as WidgetControl to the map
        metadata_control = WidgetControl(widget=self.metadata_output, position='bottomright', transparent_bg=True)
        self.map_.add_control(metadata_control)
        
        # Add controls in this way to make the new one as first in the list
        self.map_.controls = tuple([WidgetControl(widget=self.reload_btn, position='topright', transparent_bg=True)] + 
            [c for c in self.map_.controls])
        
        self.w_state_bar = StateBar(done=True)
        
        # Add controls in this way to make the new one as first in the list
        self.map_.controls = tuple([WidgetControl(widget=self.w_state_bar)]+[c for c in self.map_.controls])

        self.w_run = sw.Btn("Get Alerts")
        self.w_api_alert = Alert(children=[cm.ui.default_api], type_='info').show()
        su.hide_component(self.w_countries)
        
        # Events
        
        self.w_countries.observe(self.add_country_event, 'v_model')
        self.w_aoi_method.observe(self.aoi_method_event, 'v_model')
        self.w_alerts.observe(self.alert_list_event, 'v_model')
        self.w_alerts.observe(self.filter_confidence, 'confidence')
        
        self.w_api_btn.on_event('click', self.validate_api_event)
        self.reload_btn.on_click(self.add_layers)
        
        
        self.map_.dc.on_draw(self.handle_draw)
        self.w_run.on_event('click', self._get_alerts)
        
        # Map events
        
        self.map_.on_interaction(self._return_coordinates)
        
        # Links
        
        link((self.w_api_key, 'v_model'),(self, 'api_key'))
        link((self.w_spantime, 'v_model'),(self, 'timespan'))
        link((self.w_days_before, 'v_model'),(self, 'days_before'))
        link((self.w_max_images, 'v_model'),(self, 'max_images'))
        link((self.w_cloud_cover, 'v_model'),(self, 'cloud_cover'))
        

        # View
        
        self.opt_panel = v.Card(
            class_='pa-2 mb-2',
            children=[
                v.CardTitle(children=['Alerts settings']),
                self.w_spantime,
                self.w_aoi_method,
                self.w_countries,
                self.w_run,
                self.w_alert,
            ],)
        
        self.planet_opt = v.Card(
            class_='pa-2',
            children=[
                v.CardTitle(children=[cm.ui.planet_title]),
                v.Flex(class_='d-flex align-center mb-2', 
                       row=True, 
                       children =[self.w_api_key, self.w_api_btn]
                      ),
                self.w_api_alert, 
                self.w_max_images,
                self.w_days_before,
                self.w_cloud_cover,
            ]
        )
        
        self.children = (
            
            # Left flex options panel
            v.Flex(
                xs3 =True, 
                children =[
                    self.opt_panel,
                    self.planet_opt,
            ]),
            
            # Right flex for map
            v.Flex(
                class_='ml-2', 
                xs9 = True, 
                children =[
                    self.w_alerts,
                    self.map_
            ])
        )
        
    def filter_confidence(self, change):
        """Filter alert list by confidence"""
        
        confidence = change['new'].lower()
        
        if confidence != 'All':
            self.w_alerts.items = self.aoi_alerts[self.aoi_alerts.confidence==confidence].index.to_list()

    def _get_metadata(self, alert_id):
        """Awful way to get a metadata table of alert and display it 
        within self.map_ as control
        
        """
        col_names = ['latitude','longitude','acq_date','acq_time','confidence']
        headers= [f'{col_name.capitalize()}: ' for col_name in col_names]
        
        values=self.aoi_alerts.loc[alert_id, col_names].to_list()
        values=[round(val,2) if isinstance(val, np.float64) else val for val in values]
        
        ui.aoi_alerts.loc[ui.current_alert,]
        confidence = {'low':'red', 'high':'green', 'nominal':'orange'}

        data='<tbody>'
        for header, value in zip(headers, values):
            if header=='Confidence:':
                color = confidence[values[4]]
                print(color)
                data+=f'<tr><th>{header}</th><td><v-chip small color={color}>{value}</v-chip></td></tr>'
            else:
                data+=f'<tr><th>{header}</th><td style="font-size:90%">{value}</td></tr>'
        data+='</tbody>'

        html=f"""
          <v-simple-table dense>
                {data}
          </v-simple-table>
        """
        class Table(v.VuetifyTemplate):

            template = Unicode(html).tag(sync=True)

        with self.metadata_output:
            self.metadata_output.clear_output()
            display(v.Card(width='200px', children=[Table()]))
                    
    def _return_coordinates(self, **kwargs):

        if kwargs.get('type') == 'click':

            # Remove markdown if there is one
            remove_layers_if(self.map_, 'type', equals_to='manual', _metadata=True)

            self.lat, self.lon = kwargs.get('coordinates')

            marker = Marker(location=kwargs.get('coordinates'), 
                            alt='Manual', 
                            title='Manual', 
                            draggable=False,
                            name='Manual marker'
                           )
            marker.__setattr__('_metadata', {'type':'manual', 'id': None})

            self.map_.add_layer(marker)
        
    def _toggle_planet_setts(self, on=True):
        
        if on:
            self.w_days_before.disabled = False
            self.w_cloud_cover.disabled = False
            self.w_max_images.disabled = False
            
        else:
            self.w_days_before.disabled = True
            self.w_cloud_cover.disabled = True
            self.w_max_images.disabled = True
        
    def _get_items(self):

        geom = json.loads(dumps(Point(ui.lon, ui.lat).buffer(0.001, cap_style=3)))
        
        # Get the current year/month/day
        now = datetime.datetime.now(tz=pytz.timezone('UTC'))
        
        days_before = ([x[1] for x in list(zip(self.TIME_SPAN,[1,2,7],)) if self.timespan == x[0]])[0]
        days_before += self.days_before
        start_date = now-datetime.timedelta(days=days_before)
        req = build_request(geom, start_date, now, cloud_cover=self.cloud_cover/100)
        items = get_items('Alert', req, self.client)
        
        return items
    
    def _prioritize_items(self):
        
        self.w_state_bar.add_msg(cm.ui.searching_planet, done=False)
        
        items = self._get_items()
        items = [(item['properties']['item_type'], 
                  item['id'],
                  pd.to_datetime(item['properties']['acquired']).strftime('%Y-%m-%d-%H:%M')
                 ) for item in items[1]]
        
        items_df = pd.DataFrame(data=items, columns=['item_type', 'id', 'date'])
        items_df.sort_values(by=['item_type'])
        items_df.drop_duplicates(subset=['date', 'id'])
        
        # If more than one day is selected, get one image per day.
        
        if self.days_before:
            items_df.date = pd.to_datetime(items_df.date)
            items_df = items_df.groupby(
                [items_df.date.dt.year, items_df.date.dt.day]
            ).nth(1).reset_index(drop=True)
            
        if self.max_images:
            items_df = items_df.head(self.max_images)
        
        if len(items_df) == 1:
            self.w_state_bar.add_msg(cm.ui.one_image.format(len(items_df)), done=True)
        elif len(items_df):
            self.w_state_bar.add_msg(cm.ui.number_images.format(len(items_df)), done=True)
        else:
            self.w_state_bar.add_msg(cm.ui.no_planet, done=True)
        
        return items_df

    def add_layers(self, event=None):
        """Search planet imagery and add them to self.map_"""
        
        # Validate whether Planet API Key is valid,
        # and if there is already selected coordinates.
        
        if self.validate_state_bar(): 
        
            items_df = self._prioritize_items()

            # remove all previous loaded assets

            remove_layers_if(self.map_, 'attribution', 'Planet')

            for i, row in items_df.iterrows():
                layer = TileLayer(
                    url=f'https://tiles0.planet.com/data/v1/{row.item_type}/{row.id}/{{z}}/{{x}}/{{y}}.png?api_key={self.api_key}',
                    name=f'{row.item_type}, {row.date}',
#                     max_zoom=15,
                    attribution='Planet'
                )
                layer.__setattr__('_metadata', {'type':row.item_type, 'id':row.id})
                if row.id not in [layer._metadata['id'] for layer in self.map_.layers if hasattr(layer, '_metadata')]:
                    self.map_+layer
    
    def validate_state_bar(self):
        
        if not self.valid_api:
            self.w_state_bar.add_msg(cm.ui.no_key, done=True)
            
        elif not all((self.valid_api, self.lat, self.lon)):
            self.w_state_bar.add_msg(cm.ui.no_latlon, done=True)
            
        else:
            return True            

                        
    def validate_api_event(self, widget, change, data):
        
        api_key = self.w_api_key.v_model
        
        planet_key = PlanetKey(api_key)
        self.client = planet_key.client()
        
        self.valid_api = planet_key.is_active()
        
        if self.valid_api:
            self.w_api_alert.add_msg(cm.ui.success_api.msg, cm.ui.success_api.type)
            self._toggle_planet_setts(on=True)
        else:
            self.w_api_alert.add_msg(cm.ui.fail_api.msg, cm.ui.fail_api.type)
            self._toggle_planet_setts(on=False)
            
    def remove_layers(self):
        
        # get map layers
        layers = self.map_.layers
        
        # loop and remove layers 
        [self.map_.remove_last_layer() for _ in range(len(layers))]
        
        
    def handle_draw(self, target, action, geo_json):
        
        self.remove_layers()
        if action == 'created':
            self.aoi = geo_json['geometry']
    
    def alert_list_event(self, change):
        """ Update map zoom, center when selecting an alert
        and add metadata to map
        
        """
        
        # Get fire alert id
        self.current_alert = change['new']
        
        # Filter dataframe to get lat,lon
        
        self.lat = self.aoi_alerts.loc[self.current_alert]['latitude']
        self.lon = self.aoi_alerts.loc[self.current_alert]['longitude']
        
        self.map_.center=((self.lat,self.lon))
        self.map_.zoom=15
        self._get_metadata(self.current_alert)
        
        # Search and add layers to map
        if self.valid_api: self.add_layers()
        
        
    def aoi_method_event(self, change):
        
        self.remove_layers()
        
        if change['new'] == 'Select country':
            self.map_.hide_dc()
            su.show_component(self.w_countries)
            
        else:
            su.hide_component(self.w_countries)
            self.map_.show_dc()
            
    
    def add_country_event(self, change):
        
        self.remove_layers()
        
        country_df = COUNTRIES[COUNTRIES['name']==change['new']]
        geometry =  country_df.iloc[0].geometry
        
        lon, lat = [xy[0] for xy in geometry.centroid.xy]
        
        data = json.loads(country_df.to_json())
        
        aoi = GeoJSON(data=data,
                      name=change['new'], 
                     style={
                         'color': 'green',
                         'fillOpacity': 0, 
                         'weight': 3
                     }
                )
            
        self.aoi = aoi.data['features'][0]['geometry']
        
        min_lon, min_lat, max_lon, max_lat = geometry.bounds

        # Get (x, y) of the 4 cardinal points
        tl = (max_lat, min_lon)
        bl = (min_lat, min_lon)
        tr = (max_lat, max_lon)
        br = (min_lat, max_lon)
        
        self.map_.zoom_bounds([tl,bl, tr, br])
        self.map_.center = (lat, lon)
        self.map_.add_layer(aoi)


    def validate_inputs(self):
        
        if not self.aoi:
            self.w_alert.add_msg(cm.ui.valid_aoi,type_='error')
            self.restore_widgets()

            raise
    
    def restore_widgets(self):
        
        self.w_run.disabled=False
        self.w_run.loading=False
        self.w_alerts.items = []
        self.w_alerts.v_model = None

    def _get_url(self, satellite):
        
        satellites = {
            'viirs': ('SUOMI_VIIRS_C2', 'suomi-npp-viirs-c2'),
            'modis': ('MODIS_C6', 'c6'),
            'viirsnoa': ('J1_VIIRS_C2', 'noaa-20-viirs-c2'),
        
        }
        
        sat = satellites[satellite]
        timespan = self.timespan.replace(' hours', 'h').replace(' days','d')
        
        url=f"https://firms.modaps.eosdis.nasa.gov/data/active_fire/{sat[1]}/csv/{sat[0]}_Global_{timespan}.csv"
        return url
        
    def _get_alerts(self, widget, change, data):
        
        self.validate_inputs()
        widget.toggle_loading()
        
        self.w_alert.add_live_msg(cm.ui.downloading_alerts, type_='info')
        
        url = self._get_url('viirs')
        
        df = pd.read_csv(url)
        alerts_gdf = gpd.GeoDataFrame(df, 
                                      geometry=gpd.points_from_xy(df.longitude, 
                                                                  df.latitude), 
                                      crs="EPSG:4326")
        
        self.alerts = alerts_gdf
        
        self.aoi_alerts = self._clip_to_aoi()

        self.w_alert.add_msg(
            cm.ui.alert_number.format(len(self.aoi_alerts), self.timespan), 
            type_='success')
        
        alert_list_item = list(self.aoi_alerts.index)
        self.w_alerts.items = alert_list_item
                
        # Convert alert's geometries to 54009 (projected crs) and use 375m as buffer 
        geometry_col = ui.aoi_alerts.to_crs('EPSG:3116')['geometry'].buffer(187.5, cap_style=3).copy()
        self.aoi_alerts = ui.aoi_alerts.assign(geometry=geometry_col)
        json_aoi_alerts = json.loads(ui.aoi_alerts.to_crs('EPSG:4326').to_json())
        
        json_aoi_alerts = GeoJSON(data=json_aoi_alerts,
                                name='Alerts', 
                                style={                                    
                                     'color': 'red', 
                                     'fillOpacity': 0.1, 
                                     'weight': 2
                                 },
                                hover_style={
                                    'color': 'white', 
                                    'dashArray': '0', 
                                    'fillOpacity': 0.5
                                },)
        
        
        self.map_+json_aoi_alerts
        self.w_alerts.disabled = False
        widget.toggle_loading()
    
    def _clip_to_aoi(self):
        
        # Clip alerts_gdf to the selected aoi
        ""
        self.w_alert.add_live_msg(msg=cm.ui.clipping,type_='info')
        
        clip_geometry = Polygon(self.aoi['coordinates'][0])
        
        alerts = self.alerts[self.alerts.geometry.intersects(clip_geometry)]
        
        return alerts
    
    def _workspace(self):
        """ Creates the workspace necessary to store and manipulate the module

        return:
            Returns environment Paths

        """

        base_dir = Path(os.path.join('/home', self.USER))

        root_dir = base_dir/'Planet_fire_explorer'
        data_dir = root_dir/'data'

        root_dir.mkdir(parents=True, exist_ok=True)
        data_dir.mkdir(parents=True, exist_ok=True)

        return root_dir, data_dir

In [92]:
ui = UI()

In [94]:
ui.aoi_alerts[ui.aoi_alerts.confidence=='low'].index.to_list()

[26696,
 26698,
 26700,
 26722,
 26728,
 26732,
 26733,
 26744,
 26746,
 26749,
 26752,
 26759,
 26760,
 26762,
 26764,
 26765,
 26773,
 26784,
 26792,
 26797,
 26805,
 26810,
 26811,
 26819,
 26820,
 26827,
 26829,
 26832,
 26836,
 26837,
 26859,
 26865,
 26866,
 26895,
 26906,
 26978,
 26979,
 27012,
 27015,
 27016,
 27028,
 27036,
 27037,
 27145,
 27151,
 27152,
 27156,
 27159,
 27171,
 27180,
 27185,
 27190,
 27216,
 27234,
 27543,
 28208,
 28591,
 28800,
 28819,
 28886,
 28926,
 28935,
 28989,
 29104,
 29146,
 29151,
 29161,
 29162,
 29164,
 29167,
 29170,
 29171,
 29177,
 29180,
 29182,
 29183,
 29184,
 29187,
 29189,
 29203,
 29204,
 29273,
 29303,
 29310,
 29313,
 29324,
 29347,
 29369,
 29378,
 29389,
 29390,
 29433,
 29437,
 29438,
 29450,
 29458,
 64763,
 64871,
 64884,
 64907,
 64979,
 64986,
 65083,
 65148,
 65165,
 65176,
 65187,
 65189,
 65203,
 65204,
 65205,
 65217,
 65231,
 65237,
 65258,
 65259,
 65260,
 65269,
 65273,
 65278,
 65281,
 65335,
 65350,
 65351,
 65354,


In [93]:
ui

UI(children=[Flex(children=[Card(children=[CardTitle(children=['Alerts settings']), Select(items=['24 hours', …

In [ ]:
# process_tile = sw.Tile(id_='ui', title='Planet active fires explorer', inputs=[ui])

# appBar = sw.AppBar(title='Planet active fires explorer')

# content = [
#     process_tile,
# ]

# #create a drawer 
# item_process = sw.DrawerItem('Map', 
#                            'mdi-map-marker-check', 
#                            card="ui").display_tile(content)

# code_link = 'https://github.com/ingdanielguerrero/planet_active_fires_explorer'
# wiki_link = 'https://github.com/ingdanielguerrero/planet_active_fires_explorer/blob/main/README.md'
# issue = 'https://github.com/ingdanielguerrero/planet_active_fires_explorer/issues/new'

# items = [
#     item_process,
# ]

# drawer = sw.NavDrawer(items, 
#                       code = code_link, 
#                       wiki = wiki_link, 
#                       issue = issue, 
#                       mini_variant=True).display_drawer(appBar.toggle_button)

# #build the app 
# app = sw.App(
#     tiles=content, 
#     navDrawer=drawer
# ).show_tile('ui')
# #display the app
# app